# Exploring UC1004 dataset

In [1]:
import sys
sys.path.insert(0, './utils/')

import geo_conv
import sqlite3 as sqlite
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse
from scipy.io import mmwrite, mmread
import scipy.spatial.distance
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import KNeighborsRegressor
import folium
import datetime
import os

plt.style.use('ggplot')
%matplotlib inline

Database schema: network, us_pos, us_signal

In [2]:
def add_lat_lon(df):
    xy = zip(df.x.values, df.y.values)
    latlon = map(lambda (coord):geo_conv.convert_GK_to_lat_long(coord[0]+__XMIN, coord[1]+__YMIN), xy)
    
    if 'lat' not in df.columns:
        df = pd.concat([df, 
                        pd.DataFrame.from_records(latlon, columns=['lat', 'lon'])],
                       axis=1)
    else:
        df[['lat', 'lon']] = pd.DataFrame.from_records(latlon, columns=['lat', 'lon'])
        
    return df


def bs_mean(vector, num_samples, conf):
    boot_sample = np.zeros_like(vector)
    boot_means = np.zeros((num_samples))
    for s in xrange(num_samples):
        boot_sample = np.random.choice(vector,
                               replace=True)
        boot_means[s] = boot_sample.mean()
    boot_mean.sort()
    return np.percentile(boot_means, [(100-conf)/2, (100+conf)/2])

In [9]:
__CITY_LAT = 52.3667
__CITY_LON = 9.7167
__XMIN = 4336000
__YMIN = 5794000
__LOAD_TRAIN_TEST_MATRIX = 0
__TRAIN_TEST_DATE = ''
__M_FILE_FOLDER = './data/uc1004/mmfiles/'
__DATE = '_' + datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [10]:
# Check if folder with outputs exists
if ~os.path.isdir(__M_FILE_FOLDER):
    os.makedirs(__M_FILE_FOLDER)

In [11]:
# Setting conntection to sqlite db
conn = sqlite.connect('./data/uc1004/uc1004.db')

## Getting Networkd data

In [ ]:
# Get Network data file
query = 'SELECT * FROM network;'

network = pd.read_sql(query, conn)
network.shape

In [ ]:
network = add_lat_lon(network)
network.set_index('cell_id', inplace=True)
network.head(10)

## Getting User Position

As it goes from the article, users are categorized to:<br\>
Vehicular: 0 ... 4619 <br\>
Pedestrian: 10000 ... 15246<br\>
Static Indoor: 20000 ... 29999

In [ ]:
user_id = 10003
query = 'SELECT * FROM us_pos INDEXED BY user_index WHERE userid=%d' % user_id
user_pos = pd.read_sql(query, conn)

In [ ]:
print user_pos.shape
user_pos.sort_values('time', inplace=True)
user_pos.head()

In [ ]:
# Adding lat, long columns
user_pos = add_lat_lon(user_pos)
user_pos.head()

## Getting User's RSPS

In [ ]:
query = 'SELECT * FROM us_signal INDEXED BY user_index_sig WHERE userid=%d' % user_id
user_signals = pd.read_sql(query, conn)

In [ ]:
print user_signals.shape
user_signals.sort_values('time', inplace=True)
user_signals.head()

In [ ]:
# Identify cell set for user
cell_id_cols = range(2, 42, 2)
cell_ids = np.unique(user_signals.iloc[:, cell_id_cols].values.astype(int))

## Plotting Maps From Dataset

### Land-case map

In [ ]:
# Land-case map
use_map = np.genfromtxt('./data/uc1004/Land_Use_Map.csv', delimiter=';')

In [ ]:
print use_map.shape # It differes from info in the article
print np.unique(use_map)

In [ ]:
# .T to have refference with pics in the article
# But map shoulb be rotated by 90 degrees (according to google maps)
# Moreover, the map in data is not the same as in the article..
# fig = plt.figure(figsize=(10, 20))
# plt.imshow(use_map.T, cmap='flag')

In [ ]:
use_map = np.rot90(use_map)

In [ ]:
fig = plt.figure(figsize=(10, 20))
plt.imshow(use_map, cmap='flag')

### Best server map

In [ ]:
best_server = np.genfromtxt('./data/uc1004/Best_Server_Map.csv', delimiter=';')

In [ ]:
print best_server.shape # It differes from info in the article
print np.unique(best_server) # should be at least = to # of cells

In [ ]:
best_server = np.rot90(best_server)

In [ ]:
fig = plt.figure(figsize=(10, 20))
plt.imshow(best_server, cmap='flag')

## Generating map with user route and cell positions

In [ ]:
map_hannover = folium.Map(location=[__CITY_LAT, __CITY_LON], 
                          zoom_start=15)
map_hannover.line(locations=user_pos[['lat', 'lon']].values,
                  line_opacity=0.5, 
                  popup=('user: ' + str(user_id)))
for cid in cell_ids:
    cell = network.loc[cid]
    map_hannover.polygon_marker(location=[cell.lat, cell.lon],
                                num_sides=3,
                                radius=5,
                                rotation=cell.azimuth,
                                popup='cell_id: ' + str(cid) + '\nazimuth: ' + str(cell.azimuth))

map_hannover.create_map(path='./data/uc1004/map.html')

## Looking at RSS

In [ ]:
signals_long = (pd.wide_to_long(user_signals, ['cell_id', 'rsr'], i='time', j='cell_order')
                  .reset_index()
                  .set_index(['cell_id', 'time'])
                  .sort_index(level=0))
signals_long.head()

In [ ]:
cell_id = 6
signals_long.loc[cell_id].plot(y='rsr'
                              )
fig = plt.gcf()
fig.set_size_inches(20,10)
plt.ylabel('RSR')
plt.xlabel('time')

## Plotting Distance vs RSR

In [ ]:
# Picking cell and its RSR
cell_id = 152
cell_ss = signals_long.loc[cell_id].reset_index(drop=False)

# Considering only cases when it is in top-2 of RSR rating
cell_ss = cell_ss[cell_ss.cell_order <= 2]

# Cell info
cell_info = network.loc[cell_id]
user_dist = np.sqrt((user_pos.x - cell_info.x)**2 + (user_pos.y - cell_info.y)**2)

# Considering relevant timestamps
idx = user_pos.time.isin(cell_ss.time)
user_dist = user_dist.ix[idx]

In [ ]:
# Plotting

fig = plt.figure(figsize=(20,10))
plt.plot(user_dist, cell_ss.rsr, '.')
plt.xlabel('Distance to user %d' % user_id)
plt.ylabel('RSR from cell %d' % cell_id)

# k-NN baseline

### Train\test first look

Picking first 1000 users' positions and their rsr <br/>
First 500 going to training set. Others - to test <br/>
Distance measure - cosine distance

In [12]:
# # So slow..
# query = '''SELECT * FROM us_pos INDEXED BY user_index 
#            WHERE userid >= 0 AND userid <= 999;'''

# chunker = pd.read_sql(query, conn, chunksize=10**5)
# df_user_pos = pd.DataFrame()
# for chunk in chunker:
#     df_user_pos = df_user_pos.append(chunk)


# Export to csv is still slow, but import is rather fast..
col_types = {'time': np.dtype(float),
             'userid': np.dtype(int),
             'x': np.dtype(float),
             'y': np.dtype(float)}

df_user_pos = pd.read_csv('./data/uc1004/us_pos_exper.csv', 
                          header=0,
                          dtype=col_types,
                          usecols=col_types.keys())

In [13]:
# I have a strong feeling, that my computer's memory will be overfilled...
df_user_pos.shape

(5435392, 4)

In [14]:
# # So Slow..
# query = '''SELECT * FROM us_signal INDEXED BY user_index_sig 
#            WHERE userid >= 0 AND userid <= 999;'''

# chunker = pd.read_sql(query, conn, chunksize=10**5)
# for chunk in chunker:
#     df_user_rsr = df_user_rsr.append(chunk)
# df_user_rsr.shape

# # Manual transform to long format:
# fout = open('./data/uc1004/us_sig_long.csv', 'w')
# with open('./data/uc1004/us_sig_exper.csv', 'r') as fin:
#     fin.readline() # skip header line
#     fout.write('time,userid,cellid,cell_order,rsr\n')
#     for line in fin:
#         line = line.split(',')
#         time = line[0]
#         userid = line[1]
#         for order in xrange(1,21):
#             cellid = line[2+2*(order-1)]
#             rsr = line[1+2*(order)]
#             fout.write('%s,%s,%s,%s,%s\n' % (time, userid, cellid, order, rsr))
# fout.close()

# # Overflow...
# # Exportin long format
# col_types = {'time': np.dtype(float),
#              'userid': np.dtype(int),
#              'cellid': np.dtype(int),
#              'order': np.dtype(int),
#              'rsr': np.dtype(float)}
    
# df_user_rsr = pd.read_csv('./data/uc1004/us_sig_long.csv', 
#                           header=0,
#                           dtype=col_types)

# The only reasonably fast method and 
# memory-save method
col_types = {'time': np.dtype(float),
             'userid': np.dtype(int)}
for i in xrange(1, 21):
    col_types['cell_id%d' % i] = np.dtype(int)
    col_types['rsr%d' % i] = np.dtype(float)
    
df_user_rsr = pd.read_csv('./data/uc1004/us_sig_exper.csv', 
                          header=0,
                          dtype=col_types)

In [15]:
df_user_rsr.set_index(['userid', 'time'], inplace=True)
df_user_rsr.shape

(5435392, 40)

In [ ]:
# Lets look at the roots in train\test
df_user_pos.set_index('userid',  inplace=True)

In [ ]:
map_hannover = folium.Map(location=[__CITY_LAT, __CITY_LON], 
                          zoom_start=15)

for index, user_data in df_user_pos.groupby(level=0):
    user_data = add_lat_lon(user_data.reset_index())
    user_data.sort_values('time', inplace=True)
    if index < 500:
        map_hannover.line(locations=user_data[['lat', 'lon']].values,
                          line_opacity=0.2,
                          line_color='blue')
    else: 
        map_hannover.line(locations=user_data[['lat', 'lon']].values,
                          line_opacity=0.2,
                          line_color='red')

# Careful, ~200mb
map_hannover.create_map(path='./data/uc1004/test_train_map.html')

There is an opinion, that it is better to make random train\test split. According to the map some areas more filled with train, rather than test.

### Building obj-feat matrix

So, for each position we have to build a feature vector, which is an average signal strength for this position from all the cells

In [ ]:
df_user_pos.reset_index(inplace=True, drop=False)

# # remove floatting point
# df_user_pos.x = df_user_pos.x.astype(int)
# df_user_pos.y = df_user_pos.y.astype(int)

df_user_pos.set_index(['x','y'], inplace=True)
df_user_pos.sort_index(inplace=True)
idx_train = df_user_pos.userid < 500

# Helpful stuff
rename_dict = {}
for i in xrange(1,21):
    rename_dict['cell_id%d'%i] = 'cell_id'
    rename_dict['rsr%d'%i] = 'rsr'

In [ ]:
# Something to init
train_pos_num = len(df_user_pos[idx_train].index.unique())
test_pos_num = len(df_user_pos[~idx_train].index.unique())
pos_train = np.empty((train_pos_num,2))
pos_test = np.empty((test_pos_num,2))

cell_num = 195 # actually there are less in the data..

X_train = dok_matrix((train_pos_num, cell_num))
X_test = dok_matrix((test_pos_num, cell_num))

In [ ]:
# Filling train matrix
if __LOAD_TRAIN_TEST_MATRIX:
    filename = 'X_train' + __TRAIN_TEST_DATE + '.mm'
    X_train = mmread(os.path.join(__M_FILE_FOLDER, filename))
    filename = 'pos_train' + __TRAIN_TEST_DATE + '.mm'
    pos_train = mmread(os.path.join(__M_FILE_FOLDER, filename))
else:
    ind = 0
    for pos, pos_data in df_user_pos[idx_train].groupby(level=[0,1]):
        pos_train[ind, :] = pos

        idx = zip(pos_data.userid.values, np.round(pos_data.time.values, 1))
        sig_data = df_user_rsr.loc[idx].dropna()

        if sig_data.shape[0]:
            avg = (pd.concat([sig_data[['cell_id%d'%i,'rsr%d'%i]].rename(columns=rename_dict) for i in xrange(1,21)],
                             axis=0, ignore_index=True)
                     .groupby('cell_id')
                     .rsr.agg(np.nanmean))
            X_train[ind, avg.index.values] = avg.values

        ind+=1
        if np.mod(ind, 50000)==0:
            print ind
    filename = 'X_train' + __DATE
    mmwrite(os.path.join(__M_FILE_FOLDER, filename), X_train)
    filename = 'pos_train' + __DATE
    mmwrite(filename, pos_train)

In [ ]:
# Filling test matrix
if __LOAD_TRAIN_TEST_MATRIX:
    filename = 'X_test' + __TRAIN_TEST_DATE + '.mm'
    X_test = mmread(os.path.join(__M_FILE_FOLDER, filename))
    filename = 'pos_test' + __TRAIN_TEST_DATE + '.mm'
    pos_test = mmread(os.path.join(__M_FILE_FOLDER, filename))
else:
    ind = 0
    for pos, pos_data in df_user_pos[~idx_train].groupby(level=[0,1]):
        pos_test[ind, :] = pos

        idx = zip(pos_data.userid.values, np.around(pos_data.time.values, 1))
        sig_data = df_user_rsr.loc[idx].dropna()

        if sig_data.shape[0]:
            avg = (pd.concat([sig_data[['cell_id%d'%i,'rsr%d'%i]].rename(columns=rename_dict) for i in xrange(1,21)],
                             axis=0, ignore_index=True)
                     .groupby('cell_id')
                     .rsr.agg(np.nanmean))
            X_test[ind, avg.index.values] = avg.values       

        ind+=1
        if np.mod(ind, 50000)==0:
            print ind  

    filename = 'X_test' + __DATE
    mmwrite(os.path.join(__M_FILE_FOLDER, filename), X_test)
    filename = 'pos_test' + __DATE
    mmwrite(os.path.join(__M_FILE_FOLDER, filename), pos_test)
    
    # cleaning before knn
    del df_user_rsr
    del df_user_pos

## KNN itself

Ok, there was some error, lets look at least to what we have

In [ ]:
# Sklean KNN
knn = KNeighborsRegressor(n_neighbors=1, algorithm='brute', 
                          metric='cosine', n_jobs=-1)
knn.fit(X_train, pos_train)
pos_pred = np.empty_like(pos_test)

In [ ]:
# Not to overfill memory..
istep = 10
for i in xrange(istep, pos_pred.shape[0], istep):
    pos_pred[i-istep:i] = knn.predict(X_test[i-istep:i, :])
    
if i < pred_idx.shape[0]:
    pos_pred[i:] = knn.predict(X_test[i:, :])

In [ ]:
filename = 'pos_pred' + __DATE
mmwrite(os.path.join(__M_FILE_FOLDER, filename), pos_pred)

## Estimate accuracy

In [ ]:
err = ((pos_pred - pos_test)**2).sum(axis=1)

In [ ]:
bs_mean(err, 10000, 67)